In [ ]:
import numpy as np
import pandas as pd
from pandas.plotting._matplotlib.style import get_standard_colors
import matplotlib.pyplot as plt
from patsy import dmatrices
import statsmodels.api as sm

from src.evb import get_evb_incidence
from src.rki import get_consultations

In [ ]:
evb = get_evb_incidence().set_index('Kalenderwoche').rename(columns={ 'EVB': 'incidence' })
evb['group_is_treatment'] = 1
consultations = get_consultations().set_index('Kalenderwoche').rename(columns={ 'Konsultationen': 'incidence' })
consultations['group_is_treatment'] = 0

In [ ]:
def model_did(start, end, intervention, plot=False):
    df = pd.concat([
        evb.loc[pd.to_datetime(start):pd.to_datetime(end)],
        consultations.loc[pd.to_datetime(start):pd.to_datetime(end)]
    ]).reset_index()
    df['after_treatment'] = (df['Kalenderwoche'] > pd.to_datetime(intervention)).astype(int)
    y_train, X_train = dmatrices('incidence ~ group_is_treatment + after_treatment + group_is_treatment * after_treatment', df, return_type='dataframe')
    did_model = sm.OLS(endog=y_train, exog=X_train)
    did = did_model.fit()
    
    if plot:
        df['prediction'] = did.predict()

        grouped = df.set_index('Kalenderwoche').groupby('group_is_treatment')
        colors = get_standard_colors(num_colors=len(grouped))
        fig, ax = plt.subplots(figsize=(10, 5))
        axs = [ax, ax.twinx()]

        for (label, df), color, ax in zip(grouped, colors, axs):
            name = 'EVB' if label == 1 else 'Konsultationen'
            df['incidence'].plot(ax=ax, label=name, color=color)
            df['prediction'].plot(ax=ax, label='mean', color=color, alpha=0.6)

        for ax, label in zip(axs, ['Konsultationen', 'EVB']):
            ax.set_ylabel(label)
            ax.legend()

    return did

In [ ]:
start_date = pd.to_datetime('2020-W1-1', format='%G-W%V-%u')
n_weeks = int(52 * 3.5)
test_range = 2

weeks = [start_date + pd.Timedelta(i, 'W') for i in range(n_weeks)]
df = pd.DataFrame({
    'Kalenderwoche': weeks,
    'p-Wert': ([np.nan] * test_range) + [
        model_did(weeks[i - test_range], weeks[i + test_range],  weeks[i]).pvalues['group_is_treatment:after_treatment']
    for i in range(test_range, n_weeks - test_range)] + ([np.nan] * test_range)
}).set_index('Kalenderwoche')

df.plot(figsize=(10, 5), logy=True)
plt.axhline(y=0.05, color='r', linestyle='-', label='5%')
plt.legend()